# **Indonesian Food Nutrition: Dataset Development, Analysis, and Estimation Using YOLOv8n**

Notebook ini mendokumentasikan proses pengembangan sistem Food Recognition dan Estimasi Nilai Gizi Makanan Indonesia menggunakan pendekatan deep learning berbasis object detection. Proyek ini berfokus pada bagaimana model dapat mengenali berbagai jenis makanan Indonesia dari satu gambar (single-image approach) dan menghubungkannya dengan data nutrisi lokal untuk menghasilkan estimasi kalori dan komposisi gizi.

Metode utama yang digunakan adalah YOLOv8m, yang merupakan versi menengah dari keluarga YOLOv8 dengan keseimbangan antara akurasi dan efisiensi komputasi. Model ini dipilih karena mampu mendeteksi banyak objek makanan dalam satu citra sekaligus, sekaligus mempertahankan performa yang stabil.

Disusun oleh Legio 6:
- Ratu Rinjanhei Macinnes (01082230039)
- Joy ()
- Kent
- Vincent

## **Environment Setup**

Bab ini berfungsi sebagai fondasi teknis yang menjamin stabilitas dan kompatibilitas lingkungan sebelum model YOLOv8m dijalankan. Seluruh konfigurasi dilakukan untuk meminimalkan konflik dependensi dan memastikan proses pelatihan berjalan optimal baik di CPU maupun GPU. Tanpa tahap ini, risiko error saat training, seperti mismatch versi library, memory conflict, atau kegagalan CUDA, akan meningkat secara signifikan.

### 1. Downgrade dan Instalasi NumPy

In [ ]:
# Downgrade numpy agar kompatibel dengan modul yang kamu pakai
%pip install numpy==1.26.4 --force-reinstall

Bagian ini bertujuan untuk memastikan versi NumPy yang digunakan kompatibel dengan library deep learning yang akan dipakai, terutama PyTorch dan Ultralytics (YOLOv8).
Beberapa versi terbaru NumPy sering menimbulkan konflik dengan modul yang bergantung pada API lama, sehingga dilakukan downgrade ke versi stabil 1.26.4 agar lingkungan tetap konsisten dan menghindari error saat proses training dan inferensi.

### 2. Instalasi Library Utama untuk Object Detection

In [1]:
%pip install ultralytics==8.3.50 opencv-python==4.10.0.84 pandas matplotlib torch torchvision torchaudio pytorch-grad-cam==1.4.8 --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
  Using cached ultralytics-8.3.50-py3-none-any.whl.metadata (35 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pytorch-grad-cam==1.4.8 (from versions: none)
ERROR: No matching distribution found for pytorch-grad-cam==1.4.8


Bagian ini menginstal seluruh library inti yang digunakan dalam proyek, yaitu:

- **ultralytics** : library utama untuk menjalankan YOLOv8 (object detection dan training).
- **opencv-python** : digunakan untuk pemrosesan citra dan manipulasi visual.
- **torch, torchvision, torchaudio** : fondasi deep learning berbasis PyTorch untuk training dan inferensi.
- **pandas & matplotlib** : membantu analisis data dan visualisasi hasil training.
- **pytorch-grad-cam** : digunakan untuk visualisasi area fokus model (interpretabilitas model).
- **extra-index-url CUDA 12.1** : memastikan PyTorch menggunakan GPU NVIDIA dengan akselerasi CUDA untuk meningkatkan kecepatan komputasi.

Tahap ini memastikan lingkungan siap untuk proses pelatihan YOLOv8m secara optimal dan efisien.

### 3. Import Library dan Validasi Lingkungan

In [2]:
import torch, cv2, numpy as np
from ultralytics import YOLO

print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("NumPy:", np.__version__)
print("OpenCV:", cv2.__version__)

Torch: 2.3.1+cu121 | CUDA: True
NumPy: 1.26.4
OpenCV: 4.8.1


Output ini berfungsi sebagai checkpoint teknis awal untuk memastikan bahwa sistem siap digunakan sebelum masuk ke tahap preprocessing, training, dan evaluasi YOLOv8m.

## **Preprocessing**

### 1. Dataset Configuration & Class ID Remapping
Bagian ini mendefinisikan lokasi utama dataset serta file konfigurasi data.yaml yang digunakan YOLO untuk membaca daftar kelas dan struktur dataset. Dengan menentukan path ini secara eksplisit, seluruh proses pemetaan kelas dilakukan langsung terhadap dataset asli tanpa mengubah struktur folder secara manual.

Tujuannya adalah memastikan seluruh perubahan kelas tetap konsisten dan terdokumentasi secara sistematis.

Proses dimulai dengan membaca daftar kelas asli dari data.yaml, kemudian dilakukan aturan transformasi kelas yang akan diterapkan:
- **rename_map**: Digunakan untuk standarisasi penamaan kelas agar konsisten (contoh: "nasi goreng" → "Nasi Goreng").

- **merge_map**: Digunakan untuk menggabungkan dua kelas yang memiliki makna sama atau terlalu mirip secara visual (contoh: "Rujak" dilebur ke "Rujak Buah").

- **delete_classes**:Menentukan kelas yang sepenuhnya dihapus dari dataset karena tidak digunakan dalam penelitian atau dianggap mengganggu keseimbangan data.

Setelah itu, sistem membentuk ulang daftar kelas final yang telah dibersihkan dan bebas duplikasi. Selanjutnya, dibuat pemetaan ID dari indeks kelas lama ke indeks kelas baru untuk memastikan seluruh label bounding box tetap valid setelah perubahan struktur kelas. Tahap ini bertujuan menjaga integritas anotasi, mencegah kesalahan indeks, serta meningkatkan kualitas dan keterbacaan dataset sebelum digunakan dalam proses training YOLOv8m.

In [3]:
import yaml
import os
from glob import glob
import shutil

# --- PATH UTAMA ---
# Ganti path ini sesuai dengan lokasi folder "Food Dataset" Anda
DATASET_PATH = r"C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset"
YAML_PATH = os.path.join(DATASET_PATH, "data.yaml")

# --- PARAMETER PREPROCESSING ---
rename_map = {
    "nasi goreng": "Nasi Goreng",
    "pecel lele": "Pecel Lele",
    "rawon": "Rawon",
    "sate": "Sate",
}

merge_map = {
    "Rujak": "Rujak Buah" # Rujak akan dilebur ke Rujak Buah
}

delete_classes = ["Pecel Lele", "Bubur Ayam", "Ayam Goreng"]

# --- LOAD DATA ASLI ---
with open(YAML_PATH, "r") as f:
    data = yaml.safe_load(f)

original_classes = data["names"]
print(f"Kelas Asli ({len(original_classes)}): {original_classes}")

# --- PEMBENTUKAN KELAS FINAL DAN ID MAP ---

processed_names = []
for cls in original_classes:
    # 1. Rename
    name = rename_map.get(cls.lower(), cls)
    # 2. Merge
    name = merge_map.get(name, name)
    processed_names.append(name)

# 3. Hapus Duplikat dan Kelas yang dihapus
new_classes = []
for name in processed_names:
    if name not in delete_classes and name not in new_classes:
        new_classes.append(name)

print(f"Kelas Final ({len(new_classes)}): {new_classes}")

# --- PEMBENTUKAN ID MAP (Lama -> Baru) ---
id_map_fix = {}

for old_idx, old_name in enumerate(original_classes):
    # Terapkan rename dan merge
    name_temp = rename_map.get(old_name.lower(), old_name)
    name_final = merge_map.get(name_temp, name_temp)

    if name_final in delete_classes:
        # Kelas ini akan dihapus
        id_map_fix[old_idx] = None
    else:
        # Kelas dipertahankan atau di-merge, cari index barunya
        try:
            new_idx = new_classes.index(name_final)
            id_map_fix[old_idx] = new_idx
        except ValueError:
            # Seharusnya tidak terjadi jika logic di atas benar
            print(f"ERROR: Kelas {name_final} tidak ditemukan di kelas baru!")
            id_map_fix[old_idx] = None

print(f"ID Map (Lama -> Baru/None): {id_map_fix}")

Kelas Asli (21): ['Ayam Goreng', 'Bakso', 'Bubur Ayam', 'Cakwe', 'Dadar Gulung', 'Ikan Goreng', 'Klepon', 'Lontong Sayur', 'Martabak Manis', 'Mie Goreng', 'Nasi Padang', 'Rendang', 'Risol', 'Rujak', 'Rujak Buah', 'Sayur Asem', 'Soto Ayam', 'nasi goreng', 'pecel lele', 'rawon', 'sate']
Kelas Final (17): ['Bakso', 'Cakwe', 'Dadar Gulung', 'Ikan Goreng', 'Klepon', 'Lontong Sayur', 'Martabak Manis', 'Mie Goreng', 'Nasi Padang', 'Rendang', 'Risol', 'Rujak Buah', 'Sayur Asem', 'Soto Ayam', 'Nasi Goreng', 'Rawon', 'Sate']
ID Map (Lama -> Baru/None): {0: None, 1: 0, 2: None, 3: 1, 4: 2, 5: 3, 6: 4, 7: 5, 8: 6, 9: 7, 10: 8, 11: 9, 12: 10, 13: 11, 14: 11, 15: 12, 16: 13, 17: 14, 18: None, 19: 15, 20: 16}


Hasil preprocessing menunjukkan bahwa **dataset awal memiliki 21 kelas**, namun setelah dilakukan standarisasi nama, penggabungan, dan penghapusan kelas, jumlahnya **menjadi 17 kelas final**. Beberapa kelas seperti Ayam Goreng, Bubur Ayam, dan pecel lele dihapus secara sengaja karena tidak termasuk dalam ruang lingkup penelitian, sementara perbedaan penulisan seperti "nasi goreng" dan "rawon" berhasil distandarkan menjadi format yang seragam. Selain itu, kelas Rujak berhasil dilebur ke dalam Rujak Buah, mengurangi redundansi semantik yang berpotensi membingungkan model saat proses pembelajaran.

Peta ID (Old → New) menunjukkan bahwa beberapa indeks lama dipetakan ke None, yang menandakan kelas tersebut dihapus sepenuhnya dari dataset, sementara kelas lain dipetakan ulang ke indeks baru yang lebih terstruktur. Proses ini memastikan bahwa setiap bounding box tetap memiliki referensi kelas yang valid dan tidak terjadi kesalahan indeks saat training YOLOv8m.

### 2. Pembaruan File Label
Bagian ini melakukan pembersihan dan sinkronisasi file label YOLO agar sepenuhnya selaras dengan struktur kelas baru yang telah distandarisasi sebelumnya. Setiap file label pada split train, valid, dan test dipindai untuk memperbarui ID kelas berdasarkan peta id_map_fix. Bounding box dari kelas yang dihapus otomatis dieliminasi, sementara bounding box yang valid disesuaikan dengan indeks kelas baru. Jika suatu file label menjadi kosong setelah proses ini, maka file label beserta gambar pasangannya turut dihapus untuk menjaga konsistensi dataset dan mencegah data kosong masuk ke proses training.

In [4]:
# --- FUNGSI PEMBARUAN LABEL AMAN ---

def safe_process_labels(split):
    label_dir = os.path.join(DATASET_PATH, split, "labels")
    img_dir = os.path.join(DATASET_PATH, split, "images")

    # Pastikan direktori labels ada
    if not os.path.isdir(label_dir):
        print(f"Direktori labels untuk {split} tidak ditemukan di: {label_dir}")
        return

    label_files = glob(label_dir + "/*.txt")
    
    deleted_file_count = 0
    modified_file_count = 0

    print(f"\n--- Memproses Split: {split.upper()} ({len(label_files)} file) ---")

    for txt_path in label_files:
        with open(txt_path, "r") as f:
            lines = f.readlines()
        
        new_lines = []
        is_modified = False

        for line in lines:
            parts = line.strip().split()
            if not parts or len(parts) < 5:
                continue
                
            old_id = int(parts[0])
            
            # Cari ID baru
            new_id = id_map_fix.get(old_id) 
            
            if new_id is not None:
                # BBox Dipertahankan: Update ID baru
                if old_id != new_id:
                    parts[0] = str(new_id)
                    is_modified = True
                
                new_lines.append(" ".join(parts))
            else:
                # BBox Dihapus: Abaikan baris ini
                is_modified = True
        
        # JIKA label menjadi KOSONG atau BUKAN label kosong tapi ada modifikasi
        if not new_lines:
            # Hapus file label dan gambar
            os.remove(txt_path)
            img_path = os.path.join(img_dir, os.path.basename(txt_path).replace(".txt", ".jpg"))
            if os.path.exists(img_path):
                os.remove(img_path)
            deleted_file_count += 1
            
        elif is_modified:
            # Tulis ulang file label dengan BBox yang telah difilter/diperbarui ID-nya
            with open(txt_path, "w") as f:
                f.write("\n".join(new_lines) + "\n")
            modified_file_count += 1
            
    print(f"File label/gambar dihapus (label kosong): {deleted_file_count}")
    print(f"File label diupdate (difilter/ganti ID): {modified_file_count}")
    print("-" * 30)

# Jalankan proses untuk setiap split
safe_process_labels("train")
safe_process_labels("valid")
safe_process_labels("test")

# --- FINAL UPDATE data.yaml ---
data["train"] = f"{DATASET_PATH}/train/images"
data["val"] = f"{DATASET_PATH}/valid/images"
data["test"] = f"{DATASET_PATH}/test/images"
data["names"] = new_classes
data["nc"] = len(new_classes)

with open(YAML_PATH, "w") as f:
    yaml.safe_dump(data, f, sort_keys=False) # sort_keys=False menjaga urutan

print("✔ data.yaml telah diperbarui dengan path absolute dan kelas final!")
print(f"Total kelas baru (nc): {len(new_classes)}")


--- Memproses Split: TRAIN (2579 file) ---
File label/gambar dihapus (label kosong): 303
File label diupdate (difilter/ganti ID): 2276
------------------------------

--- Memproses Split: VALID (737 file) ---
File label/gambar dihapus (label kosong): 89
File label diupdate (difilter/ganti ID): 648
------------------------------

--- Memproses Split: TEST (368 file) ---
File label/gambar dihapus (label kosong): 39
File label diupdate (difilter/ganti ID): 329
------------------------------
✔ data.yaml telah diperbarui dengan path absolute dan kelas final!
Total kelas baru (nc): 17


Hasil proses menunjukkan bahwa dataset mengalami pemurnian signifikan. Pada split train, sebanyak 303 gambar dihapus karena tidak lagi memiliki label valid, sementara 2276 file lainnya diperbarui agar sesuai dengan struktur baru. Hal serupa terjadi pada valid (89 file dihapus, 648 diperbarui) dan test (39 file dihapus, 329 diperbarui). Ini mengindikasikan bahwa sebagian data awal memang didominasi oleh kelas yang kemudian dihilangkan atau di-merge, sehingga perlu dilakukan filtering agresif untuk menjaga kualitas anotasi.

Pembaruan akhir pada data.yaml memastikan bahwa path dataset dan daftar kelas telah mengikuti konfigurasi final sebanyak 17 kelas, sehingga seluruh pipeline YOLOv8m kini berjalan di atas dataset yang bersih, konsisten, dan bebas dari label usang. Secara keseluruhan, tahap ini meningkatkan integritas data, mengurangi noise pada proses training, serta mencegah kesalahan indeks yang dapat menurunkan performa model atau menyebabkan crash saat pelatihan.

In [5]:
import os
from glob import glob
import yaml
from collections import Counter

# --- VARIABEL UTAMA (Pastikan DATASET_PATH dan new_classes sudah terdefinisi) ---
DATASET_PATH = r"C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset"
YAML_PATH = os.path.join(DATASET_PATH, "data.yaml")

# Muat data.yaml terbaru untuk mendapatkan nama kelas final
with open(YAML_PATH, "r") as f:
    data_final = yaml.safe_load(f)

new_classes = data_final["names"] # Kelas hasil preprocessing
nc = data_final["nc"]

# Inisialisasi penghitung untuk semua kelas
class_counts = Counter()
total_files = 0
total_bbox = 0
splits = ["train", "valid", "test"]

print("=========================================")
print("  ANALISIS DISTRIBUSI KELAS (BBOX COUNT) ")
print("=========================================")

for split in splits:
    label_dir = os.path.join(DATASET_PATH, split, "labels")
    label_files = glob(label_dir + "/*.txt")
    total_files += len(label_files)
    
    # Penghitung per split
    split_counts = Counter()
    
    for txt_path in label_files:
        try:
            with open(txt_path, "r") as f:
                lines = f.readlines()
            
            for line in lines:
                parts = line.strip().split()
                if parts and len(parts) >= 5:
                    class_id = int(parts[0])
                    # Tambahkan ke penghitung split dan total
                    split_counts[class_id] += 1
                    class_counts[class_id] += 1
                    total_bbox += 1
        
        except Exception as e:
            print(f"Error membaca file {txt_path}: {e}")

    # Tampilkan hasil per split (Opsional, untuk detail)
    # print(f"\n--- {split.upper()} (Total File: {len(label_files)}) ---")
    # for class_id in range(nc):
    #     class_name = new_classes[class_id]
    #     count = split_counts.get(class_id, 0)
    #     print(f"  {class_id:2d} {class_name:<15}: {count}")


print("\n=========================================")
print(f"✅ HASIL FINAL (Total {nc} Kelas)")
print(f"Total File Gambar/Label Tersisa: {total_files}")
print(f"Total Bounding Box Keseluruhan: {total_bbox}")
print("=========================================")

# Tampilkan hasil total yang diformat
print("\n[ ID | Nama Kelas | Total BBox ]")
print("-----------------------------------")
for class_id in range(nc):
    class_name = new_classes[class_id]
    count = class_counts.get(class_id, 0)
    
    # Cetak dengan warna merah jika jumlahnya 0
    if count == 0:
        print(f"❌ {class_id:2d} | {class_name:<15} | {count} <-- KOSONG!")
    else:
        print(f"   {class_id:2d} | {class_name:<15} | {count}")

print("-----------------------------------")

  ANALISIS DISTRIBUSI KELAS (BBOX COUNT) 

✅ HASIL FINAL (Total 17 Kelas)
Total File Gambar/Label Tersisa: 3253
Total Bounding Box Keseluruhan: 7575

[ ID | Nama Kelas | Total BBox ]
-----------------------------------
    0 | Bakso           | 111
    1 | Cakwe           | 830
    2 | Dadar Gulung    | 1006
    3 | Ikan Goreng     | 509
    4 | Klepon          | 1625
    5 | Lontong Sayur   | 202
    6 | Martabak Manis  | 767
    7 | Mie Goreng      | 175
    8 | Nasi Padang     | 220
    9 | Rendang         | 267
   10 | Risol           | 576
   11 | Rujak Buah      | 198
   12 | Sayur Asem      | 203
   13 | Soto Ayam       | 224
   14 | Nasi Goreng     | 224
   15 | Rawon           | 239
   16 | Sate            | 199
-----------------------------------


Distribusi bounding box menunjukkan adanya ketidakseimbangan antar kelas. Beberapa kelas memiliki representasi sangat tinggi seperti Klepon (1625 bbox) dan Dadar Gulung (1006 bbox), sementara kelas lain seperti Bakso (111 bbox) dan Mie Goreng (175 bbox) memiliki jumlah yang jauh lebih rendah. Ini mengindikasikan potensi class imbalance yang dapat menyebabkan model lebih bias terhadap kelas dengan jumlah data besar saat training.

Namun, tidak ditemukan kelas dengan jumlah bounding box nol, yang berarti seluruh kelas final benar-benar memiliki data representatif dan layak untuk dipelajari oleh model. Hal ini memperlihatkan bahwa proses filtering sebelumnya berhasil menjaga integritas dataset tanpa mengorbankan keberadaan kelas penting.

### 3. Downsampling (Pengurangan Data)
Tahap ini bertujuan untuk mengurangi ketimpangan distribusi data (class imbalance) dengan cara menurunkan jumlah sampel pada kelas yang memiliki bounding box terlalu dominan. Dalam kasus ini, kelas Dadar Gulung (ID 2) dan Klepon (ID 4) dipilih karena jumlah bounding box-nya jauh lebih tinggi dibanding kelas lain. Target yang ditetapkan adalah sekitar 550 bounding box per kelas agar distribusi data menjadi lebih proporsional.

Proses dilakukan secara aman dengan mengidentifikasi file gambar yang mengandung kelas target, kemudian menghitung estimasi jumlah file yang perlu dipertahankan berdasarkan rata-rata bounding box per file. File yang melebihi batas tersebut dipilih secara acak (random sampling) dan dipindahkan ke folder excess_data, bukan dihapus permanen, sehingga dataset tetap memiliki cadangan untuk kebutuhan evaluasi atau eksperimen ulang.

In [7]:
import os
import random
from glob import glob
import shutil
import yaml
from collections import Counter

# --- KONFIGURASI ---
# Path Anda (Pastikan sudah benar)
DATASET_PATH = r"C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset"
YAML_PATH = os.path.join(DATASET_PATH, "data.yaml")

# Muat data.yaml terbaru untuk nama kelas final
with open(YAML_PATH, "r") as f:
    data_final = yaml.safe_load(f)
new_classes = data_final["names"]

# Kelas yang akan dikurangi (Berdasarkan ID di output Anda)
# ID 2: Dadar Gulung (1006 BBox)
# ID 4: Klepon (1625 BBox)
CLASSES_TO_REDUCE = [2, 4] 

# Target BBox count: Kita akan hitung target image count berdasarkan target BBox
TARGET_BBOX_COUNT = 550 # Target BBox per kelas setelah downsample

# Folder tujuan untuk menyimpan data yang dihapus (untuk keamanan)
EXCESS_DIR = os.path.join(DATASET_PATH, "excess_data")
os.makedirs(EXCESS_DIR, exist_ok=True)

# --- FUNGSI DOWN SAMPLING ---

def downsample_class_images(split="train"):
    print(f"\n--- Memulai Downsampling pada split: {split.upper()} ---")
    
    label_dir = os.path.join(DATASET_PATH, split, "labels")
    img_dir = os.path.join(DATASET_PATH, split, "images")
    label_files = glob(label_dir + "/*.txt")
    
    # Map untuk menyimpan semua file yang mengandung kelas target
    class_file_map = {cls_id: [] for cls_id in CLASSES_TO_REDUCE}
    
    # 1. IDENTIFIKASI FILE GAMBAR YANG MENGANDUNG KELAS TARGET
    current_bbox_counts = Counter()
    
    for txt_path in label_files:
        with open(txt_path, "r") as f:
            lines = f.readlines()
            
        # Set untuk mengetahui kelas apa saja yang ada di file ini
        classes_in_file = set()
        
        for line in lines:
            parts = line.strip().split()
            if parts and len(parts) >= 5:
                class_id = int(parts[0])
                current_bbox_counts[class_id] += 1
                classes_in_file.add(class_id)
        
        # Tambahkan file ke map jika mengandung kelas yang ingin dikurangi
        for cls_id in CLASSES_TO_REDUCE:
            if cls_id in classes_in_file:
                class_file_map[cls_id].append(txt_path)


    # 2. HITUNG DAN HAPUS KELEBIHAN FILE
    files_to_remove = set()
    
    for cls_id in CLASSES_TO_REDUCE:
        class_name = new_classes[cls_id]
        total_bbox = current_bbox_counts.get(cls_id, 0)
        
        if total_bbox <= TARGET_BBOX_COUNT:
            print(f"[{class_name}] OK: BBox Count {total_bbox} (<= {TARGET_BBOX_COUNT}). Tidak perlu dikurangi.")
            continue
            
        files_for_class = class_file_map[cls_id]
        
        # --- Estimasi Target File Count ---
        # Untuk menentukan berapa banyak file yang harus dihapus, kita menggunakan rasio.
        # Estimasi rata-rata BBox per file untuk kelas ini
        avg_bbox_per_file = total_bbox / len(files_for_class) if files_for_class else 1
        
        # Perkiraan jumlah file yang harus dipertahankan
        target_file_count = int(TARGET_BBOX_COUNT / avg_bbox_per_file)
        
        files_to_delete_count = len(files_for_class) - target_file_count

        if files_to_delete_count > 0:
            # Ambil sampel acak dari file yang akan dihapus
            files_to_delete = random.sample(files_for_class, files_to_delete_count)
            files_to_remove.update(files_to_delete) # Gunakan 'set' untuk menghindari duplikasi
            
            print(f"[{class_name}] REDUCE: BBox Asli {total_bbox}. Target File Keep: {target_file_count}. Menghapus {files_to_delete_count} file.")
        else:
            print(f"[{class_name}] OK: BBox Count {total_bbox}. Target file count sudah tercapai.")

    # 3. EKSEKUSI PENGHAPUSAN (Memindahkan ke folder EXCESS_DIR)
    removed_count = 0
    
    for txt_path in files_to_remove:
        # Pindahkan file label
        shutil.move(txt_path, os.path.join(EXCESS_DIR, os.path.basename(txt_path)))
        
        # Cari dan Pindahkan file gambar
        file_name_no_ext = os.path.basename(txt_path).replace(".txt", "")
        img_path = os.path.join(img_dir, f"{file_name_no_ext}.jpg") # Asumsi format .jpg
        if not os.path.exists(img_path):
            # Coba format .png jika .jpg tidak ada
            img_path = os.path.join(img_dir, f"{file_name_no_ext}.png")
            
        if os.path.exists(img_path):
            shutil.move(img_path, os.path.join(EXCESS_DIR, os.path.basename(img_path)))
        
        removed_count += 1

    print(f"\n--- Selesai Downsampling ---")
    print(f"Total file gambar/label yang dipindahkan (dihapus): {removed_count}")
    print(f"File berlebihan disimpan di: {EXCESS_DIR}")
    
# --- Jalankan Downsampling HANYA pada data 'train' ---
downsample_class_images("train")


--- Memulai Downsampling pada split: TRAIN ---
[Dadar Gulung] REDUCE: BBox Asli 781. Target File Keep: 80. Menghapus 35 file.
[Klepon] REDUCE: BBox Asli 1085. Target File Keep: 70. Menghapus 70 file.

--- Selesai Downsampling ---
Total file gambar/label yang dipindahkan (dihapus): 105
File berlebihan disimpan di: C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\excess_data


Hasilnya, pada split train:
- Dadar Gulung dikurangi dari 781 bbox dengan menghapus 35 file gambar.
- Klepon dikurangi dari 1085 bbox dengan menghapus 70 file gambar.

Total terdapat 105 file yang dipindahkan dari dataset utama.

**Mengapa Hanya Split train?**

- Pelatihan: Data train digunakan untuk mengajarkan model. Menyeimbangkan kelas di sini akan mengurangi bias.
- Validasi/Test: Data valid dan test harus mencerminkan distribusi kelas yang sebenarnya di dunia nyata. Jika di dunia nyata Klepon lebih banyak muncul, data valid dan test harus tetap mengandung lebih banyak Klepon untuk menguji performa model secara realistis.

In [8]:
import os
from glob import glob
import yaml
from collections import Counter

# --- KONFIGURASI PATH ---
DATASET_PATH = r"C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset"
YAML_PATH = os.path.join(DATASET_PATH, "data.yaml")

# Muat data.yaml terbaru untuk mendapatkan nama kelas final
with open(YAML_PATH, "r") as f:
    data_final = yaml.safe_load(f)

new_classes = data_final["names"] 
nc = data_final["nc"]

# Inisialisasi penghitung
class_counts = Counter()
total_files = 0
total_bbox = 0
splits = ["train", "valid", "test"]

print("=========================================")
print("  ANALISIS DISTRIBUSI KELAS (BBOX COUNT) ")
print("=========================================")

for split in splits:
    label_dir = os.path.join(DATASET_PATH, split, "labels")
    label_files = glob(label_dir + "/*.txt")
    total_files += len(label_files)
    
    for txt_path in label_files:
        try:
            with open(txt_path, "r") as f:
                lines = f.readlines()
            
            for line in lines:
                parts = line.strip().split()
                if parts and len(parts) >= 5:
                    class_id = int(parts[0])
                    class_counts[class_id] += 1
                    total_bbox += 1
        
        except Exception as e:
            print(f"Error membaca file {txt_path}: {e}")

print("\n=========================================")
print(f"✅ HASIL FINAL (Total {nc} Kelas)")
print(f"Total File Gambar/Label Tersisa: {total_files}")
print(f"Total Bounding Box Keseluruhan: {total_bbox}")
print("=========================================")

# Tampilkan hasil total yang diformat
print("\n[ ID | Nama Kelas | Total BBox ]")
print("-----------------------------------")
for class_id in range(nc):
    class_name = new_classes[class_id]
    count = class_counts.get(class_id, 0)
    
    # Cetak dengan tanda seru jika jumlahnya 0 atau masih tinggi (di atas 600)
    indicator = " "
    if count == 0:
        indicator = "❌"
    elif count > 600:
        indicator = "⚠️"
        
    print(f"{indicator} {class_id:2d} | {class_name:<15} | {count}")

print("-----------------------------------")

  ANALISIS DISTRIBUSI KELAS (BBOX COUNT) 

✅ HASIL FINAL (Total 17 Kelas)
Total File Gambar/Label Tersisa: 3148
Total Bounding Box Keseluruhan: 6738

[ ID | Nama Kelas | Total BBox ]
-----------------------------------
   0 | Bakso           | 111
⚠️  1 | Cakwe           | 830
⚠️  2 | Dadar Gulung    | 777
   3 | Ikan Goreng     | 505
⚠️  4 | Klepon          | 1021
   5 | Lontong Sayur   | 202
⚠️  6 | Martabak Manis  | 767
   7 | Mie Goreng      | 175
   8 | Nasi Padang     | 220
   9 | Rendang         | 267
  10 | Risol           | 576
  11 | Rujak Buah      | 198
  12 | Sayur Asem      | 203
  13 | Soto Ayam       | 224
  14 | Nasi Goreng     | 224
  15 | Rawon           | 239
  16 | Sate            | 199
-----------------------------------


Beberapa kelas seperti Klepon (1021 bbox), Cakwe (830 bbox), Dadar Gulung (777 bbox), dan Martabak Manis (767 bbox) masih berada di atas ambang 600 bounding box dan ditandai sebagai ⚠️. Namun, kondisi ini masih tergolong aman untuk pelatihan YOLOv8, karena **rasio distribusi antar kelas masih berada dalam rentang yang bisa ditoleransi oleh mekanisme loss balancing dan optimasi bawaan YOLO**

YOLOv8 secara internal menggunakan mekanisme seperti class-aware loss weighting, IoU-based optimization, dan dynamic anchor matching, yang membuatnya cukup robust terhadap ketidakseimbangan moderat seperti ini. Dengan demikian, distribusi saat ini sudah berada pada kondisi yang stabil, di mana model tidak akan terlalu bias terhadap satu kelas tertentu, namun tetap memiliki cukup variasi data untuk mempelajari fitur visual setiap jenis makanan.

## Training YOLOv8m

In [10]:
from ultralytics import YOLO
import torch
import os

# --- KONFIGURASI PATH ---
# Ganti path ini ke lokasi file data.yaml final Anda
DATA_YAML_PATH = r"C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\data.yaml"

# Pastikan path data sudah benar
if not os.path.exists(DATA_YAML_PATH):
    raise FileNotFoundError(f"File data.yaml tidak ditemukan di: {DATA_YAML_PATH}")

# --- LOAD MODEL DAN DEVICE ---
# Load base model (medium)
print("Mengunduh dan memuat model yolov8m.pt...")
model = YOLO("yolov8m.pt")

# Tentukan Device: GPU (0) atau CPU
device = 0 if torch.cuda.is_available() else "cpu"
print(f"Menggunakan Device: {device}")

# --- PARAMETER PELATIHAN ---
# Catatan: Jika imgsz=768 dan batch=4 masih menyebabkan OOM pada GPU kecil, 
# coba kurangi imgsz menjadi 640 atau batch menjadi 2.

train_results = model.train(
    data=DATA_YAML_PATH,
    epochs=200,             # Jumlah total epochs
    patience=30,            # Early Stopping (setelah 30 epoch tanpa perbaikan)
    imgsz=768,              # Ukuran gambar input (768x768)
    batch=4,                # Batch size (Disesuaikan)
    device=device,
    
    # 🔥 Optimizer & Learning Rate strategy
    optimizer="AdamW",      # Direkomendasikan untuk fine-tuning
    lr0=0.001,              # Initial Learning Rate
    cos_lr=True,            # Menggunakan Cosine LR Scheduler (baik untuk fine-tuning)
    warmup_epochs=5,        # Warming up LR selama 5 epoch
    weight_decay=0.0005,
    
    # 🔥 Augmentations (Tidak diubah, ini sudah baik)
    hsv_h=0.012, hsv_s=0.6, hsv_v=0.35,
    degrees=4, translate=0.10, scale=0.40, shear=0.05, perspective=0.0,
    fliplr=0.5, flipud=0.15, mosaic=0.20, mixup=0.05, erasing=0.25,
    multi_scale=False,
    
    # 🔥 Logging Baru
    project="indonesian_food_yolov8", # Nama folder proyek utama
    name="yolov8m_balance_run",       # Nama folder hasil pelatihan spesifik
    exist_ok=True,
)

print("\n✔ Pelatihan dimulai!")
print(f"Hasil akan disimpan di folder: runs/detect/indonesian_food_yolov8/yolov8m_balance_run")

Mengunduh dan memuat model yolov8m.pt...
Menggunakan Device: 0
New https://pypi.org/project/ultralytics/8.3.231 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.74  Python-3.10.11 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\data.yaml, epochs=200, time=None, patience=30, batch=4, imgsz=768, save=True, save_period=-1, cache=False, device=0, workers=8, project=indonesian_food_yolov8, name=yolov8m_balance_run, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=

train: Scanning C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\train\labels.cache... 2171 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2171/2171 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 3235, len(boxes) = 4467. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\valid\labels.cache... 648 images, 0 backgrounds, 0 corrupt: 100%|██████████| 648/648 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 1100, len(boxes) = 1520. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

                   all        648       1520      0.876      0.826      0.887      0.729



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    143/200      3.31G     0.5351     0.4815       1.12         15        768: 100%|██████████| 543/543 [01:24<00:00,  6.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:09<00:00,  8.57it/s]

                   all        648       1520      0.851      0.849      0.884      0.727


PermissionError: [Errno 13] Permission denied: 'indonesian_food_yolov8\\yolov8m_balance_run\\results.csv'

In [1]:
from ultralytics import YOLO
import torch

# --- KONFIGURASI PATH ---
DATA_YAML_PATH = r"C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\data.yaml"

# PATH UNTUK LANJUTKAN DARI CHECKPOINT TERAKHIR
# Asumsikan checkpoint terakhir yang valid ada di folder run Anda
LAST_CHECKPOINT = "indonesian_food_yolov8/yolov8m_balance_run/weights/last.pt" 

# Muat model dari checkpoint terakhir)
model = YOLO(LAST_CHECKPOINT)

# GPU / CPU
device = 0 if torch.cuda.is_available() else "cpu"
print("Training dilanjutkan pada:", device)

train_results = model.train(
    data=DATA_YAML_PATH,
    epochs=200,             
    patience=30,            
    imgsz=768,              
    batch=4, 
    device=device,

    # 🔥 OPTIMASI UTAMA (Mencegah Collapse)
    optimizer="AdamW",
    lr0=0.0005,             # DITURUNKAN DARI 0.001 MENJADI 0.0005
    cos_lr=True,
    warmup_epochs=5,
    weight_decay=0.0005,

    # 🔥 AUGMENTASI (Disesuaikan agar tidak terlalu agresif)
    hsv_h=0.012, hsv_s=0.6, hsv_v=0.35,
    degrees=4, translate=0.10, scale=0.40, shear=0.05, perspective=0.0,
    fliplr=0.5, flipud=0.15, 
    mosaic=0.10,            # DITURUNKAN DARI 0.20
    mixup=0.05, 
    erasing=0.10,           # DITURUNKAN DARI 0.25

    multi_scale=False,

    # 🔥 Logging
    project="indonesian_food_yolov8",
    name="yolov8m_final_rerun", # Ganti nama folder untuk run baru
    exist_ok=True,
)

Training dilanjutkan pada: 0
New https://pypi.org/project/ultralytics/8.3.231 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.74  Python-3.10.11 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine\trainer: task=detect, mode=train, model=indonesian_food_yolov8/yolov8m_balance_run/weights/last.pt, data=C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\data.yaml, epochs=200, time=None, patience=30, batch=4, imgsz=768, save=True, save_period=-1, cache=False, device=0, workers=8, project=indonesian_food_yolov8, name=yolov8m_final_rerun, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybri

train: Scanning C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\train\labels.cache... 2171 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2171/2171 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 3235, len(boxes) = 4467. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\valid\labels.cache... 648 images, 0 backgrounds, 0 corrupt: 100%|██████████| 648/648 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 1100, len(boxes) = 1520. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

    129/200      2.76G     0.4062     0.3476      1.004          5        768: 100%|██████████| 543/543 [01:17<00:00,  7.01it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:08<00:00,  9.79it/s]

                   all        648       1520      0.869      0.845      0.882      0.742


EarlyStopping: Training stopped early as no improvement observed in last 30 epochs. Best results observed at epoch 99, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

129 epochs completed in 3.126 hours.
Optimizer stripped from indonesian_food_yolov8\yolov8m_final_rerun\weights\last.pt, 52.0MB
Optimizer stripped from indonesian_food_yolov8\yolov8m_final_rerun\weights\best.pt, 52.0MB

Validating indonesian_food_yolov8\yolov8m_final_rerun\weights\best.pt...
Ultralytics YOLOv8.2.74  Python-3.10.11 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
Model summary (fused): 218 layers, 25,849,603 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:07<00:00, 10.31it/s]


                   all        648       1520      0.893      0.818      0.881      0.744
                 Bakso         24         25      0.954       0.92      0.916      0.693
                 Cakwe         33        201      0.827      0.542      0.744      0.572
          Dadar Gulung         19        125      0.827      0.648       0.76      0.631
           Ikan Goreng         53        119       0.83      0.697      0.762      0.578
                Klepon         44        383      0.845       0.58      0.752      0.597
         Lontong Sayur         34         34      0.884      0.912      0.934      0.884
        Martabak Manis         39        177      0.947      0.713      0.858      0.703
            Mie Goreng         34         37          1      0.779       0.95      0.866
           Nasi Padang         42         43      0.913      0.971      0.942      0.816
               Rendang         56         56      0.937      0.911      0.964      0.853
                 Riso

### Extended Fine-Tuning

In [1]:
from ultralytics import YOLO
import torch

# --- KONFIGURASI PATH ---
DATA_YAML_PATH = r"C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\data.yaml"

# PATH CHECKPOINT: Muat BEST.PT dari run sebelumnya ('yolov8m_final_rerun')
# Ini adalah bobot yang menghasilkan mAP 0.72886!
PREVIOUS_BEST_WEIGHTS = "indonesian_food_yolov8/yolov8m_final_rerun/weights/best.pt" 

# Muat model dari bobot TERBAIK sebelumnya
model = YOLO(PREVIOUS_BEST_WEIGHTS)

# GPU / CPU
device = 0 if torch.cuda.is_available() else "cpu"
print("Extended Fine-Tuning pada:", device)
print(f"Memuat bobot dari: {PREVIOUS_BEST_WEIGHTS}")

# =======================================================
# 1. START EXTENDED FINE-TUNING
# =======================================================

train_results = model.train(
    data=DATA_YAML_PATH,
    epochs=50,              # Durasi pelatihan lanjutan
    patience=10,            # Patience yang lebih ketat
    imgsz=768,              
    batch=4, 
    device=device,

    # 🔥 OPTIMASI UTAMA: TURUNKAN LR SANGAT RENDAH
    optimizer="AdamW",
    lr0=0.00005,            # Learning Rate diturunkan drastis
    cos_lr=True,
    warmup_epochs=5,
    weight_decay=0.0005,

    # 🔥 AUGMENTASI 
    hsv_h=0.012, hsv_s=0.6, hsv_v=0.35,
    degrees=4, translate=0.10, scale=0.40, shear=0.05, perspective=0.0,
    fliplr=0.5, flipud=0.15, 
    mosaic=0.10, mixup=0.05, erasing=0.10,           

    multi_scale=False,

    # 🔥 Logging
    project="indonesian_food_yolov8",
    name="yolov8m_extended_ft", # Nama folder baru untuk run Extended Fine-Tuning
    exist_ok=True,
)

print("\n✔ Extended Fine-Tuning Selesai!")


Extended Fine-Tuning pada: 0
Memuat bobot dari: indonesian_food_yolov8/yolov8m_final_rerun/weights/best.pt
New https://pypi.org/project/ultralytics/8.3.231 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.74  Python-3.10.11 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
engine\trainer: task=detect, mode=train, model=indonesian_food_yolov8/yolov8m_final_rerun/weights/best.pt, data=C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\data.yaml, epochs=50, time=None, patience=10, batch=4, imgsz=768, save=True, save_period=-1, cache=False, device=0, workers=8, project=indonesian_food_yolov8, name=yolov8m_extended_ft, exist_ok=True, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True

train: Scanning C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\train\labels.cache... 2171 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2171/2171 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 3235, len(boxes) = 4467. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\valid\labels.cache... 648 images, 0 backgrounds, 0 corrupt: 100%|██████████| 648/648 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 1100, len(boxes) = 1520. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

      18/50      2.76G     0.4088     0.3416      1.013          4        768: 100%|██████████| 543/543 [01:19<00:00,  6.79it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:08<00:00,  9.49it/s]

                   all        648       1520      0.857      0.849      0.879       0.74
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 8, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



18 epochs completed in 0.456 hours.
Optimizer stripped from indonesian_food_yolov8\yolov8m_extended_ft\weights\last.pt, 52.0MB
Optimizer stripped from indonesian_food_yolov8\yolov8m_extended_ft\weights\best.pt, 52.0MB

Validating indonesian_food_yolov8\yolov8m_extended_ft\weights\best.pt...
Ultralytics YOLOv8.2.74  Python-3.10.11 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
Model summary (fused): 218 layers, 25,849,603 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 81/81 [00:08<00:00, 10.07it/s]


                   all        648       1520      0.868      0.843      0.881      0.744
                 Bakso         24         25      0.786       0.88      0.857       0.67
                 Cakwe         33        201      0.742      0.721      0.756      0.571
          Dadar Gulung         19        125      0.811      0.664      0.762      0.642
           Ikan Goreng         53        119      0.775      0.697       0.75      0.603
                Klepon         44        383      0.851      0.599      0.773       0.61
         Lontong Sayur         34         34      0.887      0.882      0.927      0.864
        Martabak Manis         39        177      0.945      0.679      0.852      0.688
            Mie Goreng         34         37      0.972      0.932      0.952      0.851
           Nasi Padang         42         43      0.864      0.953      0.968      0.827
               Rendang         56         56      0.958      0.946      0.979      0.847
                 Riso

## **Evaluasi pada Split test**

In [7]:
from ultralytics import YOLO
import torch # Diperlukan untuk mengecek device jika Anda menjalankan code di lingkungan lokal

# --- KONFIGURASI PATH ---
# GANTI PATH ke BEST.PT dari run Extended Fine-Tuning terakhir Anda
MODEL_PATH = "indonesian_food_yolov8/yolov8m_extended_ft/weights/best.pt"
DATA_YAML_PATH = r"C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\data.yaml"

# Muat model terbaik
model = YOLO(MODEL_PATH)

print("Memulai Evaluasi Resmi pada Split 'test'...")

# Jalankan evaluasi
metrics = model.val(
    data=DATA_YAML_PATH, 
    split='test', 
    imgsz=768, 
    conf=0.001,
    iou=0.7,
    # Anda juga bisa menambahkan argumen save_json=True untuk detail lebih lanjut
)

# --- EKSTRAKSI DAN CETAK METRIK DENGAN PENGKOREKSIAN VALUEERROR ---

# Ambil nilai rata-rata global yang dijamin scalar:
p = metrics.box.p.mean()      # Precision (mengambil nilai rata-rata jika p adalah array)
r = metrics.box.r.mean()      # Recall (mengambil nilai rata-rata jika r adalah array)
map50 = metrics.box.map50     # mAP50 (sudah scalar)
map95 = metrics.box.map       # mAP50-95 (sudah scalar)

# Hitung F1-score: F1 = 2 * (Precision * Recall) / (Precision + Recall)
# Pengecekan ini kini aman karena p dan r sudah dipastikan scalar (nilai tunggal).
if (p + r) > 0:
    f1_score = 2 * (p * r) / (p + r)
else:
    f1_score = 0.0

Memulai Evaluasi Resmi pada Split 'test'...
Ultralytics YOLOv8.2.74  Python-3.10.11 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070 Ti Laptop GPU, 8192MiB)
Model summary (fused): 218 layers, 25,849,603 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\test\labels.cache... 329 images, 0 backgrounds, 0 corrupt: 100%|██████████| 329/329 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 562, len(boxes) = 751. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [02:07<00:00,  6.07s/it]


                   all        329        751      0.885      0.848      0.891      0.752
                 Bakso         10         10      0.814        0.9      0.872      0.563
                 Cakwe         21        113      0.707      0.699       0.75      0.514
          Dadar Gulung         16        100      0.976       0.73      0.898      0.794
           Ikan Goreng         23         46      0.781      0.587      0.652      0.509
                Klepon         18        157      0.971      0.629      0.866      0.737
         Lontong Sayur         20         22        0.9      0.864      0.883      0.815
        Martabak Manis         19         52      0.895      0.827      0.887      0.758
            Mie Goreng         17         18      0.966      0.944       0.99      0.894
           Nasi Padang         21         21      0.986          1      0.995      0.832
               Rendang         27         32      0.907      0.781      0.809       0.73
                 Riso

In [8]:
print("\n=======================================================")
print("✅ HASIL EVALUASI FINAL PADA DATA TEST")
print("=======================================================")
print(f"Precision (p): {p:.5f}")
print(f"Recall (r):    {r:.5f}")
print(f"F1-Score:      {f1_score:.5f}")
print("-------------------------------------------------------")
print(f"mAP50:         {map50:.5f}")
print(f"mAP50-95:      {map95:.5f}")
print("=======================================================")


✅ HASIL EVALUASI FINAL PADA DATA TEST
Precision (p): 0.88513
Recall (r):    0.84761
F1-Score:      0.86596
-------------------------------------------------------
mAP50:         0.89074
mAP50-95:      0.75249
